In [1]:
#import sys
#from osgeo import gdal
import fiona
import pandas as pd

PROJ: proj_create_from_database: Cannot find proj.db


In [2]:
data_path = '../../../../Praktikum_data/geodata_vienna/'

## Export sectors upper left anf lower right coordinates

In [3]:
sectors = fiona.open(data_path + 'MZKBLATT5000OGD/MZKBLATT5000OGDPolygon.shp')
#print(sectors.next())
data_list = []
data = []
for sector in sectors:
    #print(sector['id'], sector['properties']['BNR5000'], sector['geometry']['coordinates'][0][0], sector['geometry']['coordinates'][0][10])
    data.append(sector['id'])
    data.append(sector['properties']['BNR5000'])
    
    # find min/max x/y
    min_x = sector['geometry']['coordinates'][0][0][0]
    max_x = min_x            
    min_y = sector['geometry']['coordinates'][0][0][1]
    max_y = min_y
    for idx in range(len(sector['geometry']['coordinates'][0])):
        if sector['geometry']['coordinates'][0][idx][0] < min_x:
            min_x = sector['geometry']['coordinates'][0][idx][0]
        if sector['geometry']['coordinates'][0][idx][0] > max_x:
            max_x = sector['geometry']['coordinates'][0][idx][0]
        if sector['geometry']['coordinates'][0][idx][1] < min_y:
            min_y = sector['geometry']['coordinates'][0][idx][1]
        if sector['geometry']['coordinates'][0][idx][1] > max_y:
            max_y = sector['geometry']['coordinates'][0][idx][1]
    
    # this order for East and North only!
    data.append(min_x)
    data.append(max_y)
    data.append(max_x)
    data.append(min_y)
    
    data_list.append(data)
    data = []
#print(data_list)

In [4]:
# for East and North only!
df = pd.DataFrame(data_list, columns = ['id', 'sector', 'coor_min_x', 'coor_max_y', 'coor_max_x', 'coor_min_y'])
df.to_json(data_path + r'vienna_sectors.json', orient='index')

In [5]:
sectors = pd.read_json(data_path + r'vienna_sectors.json', orient='index')
sectors

,id,sector,coor_min_x,coor_max_y,coor_max_x,coor_min_y
0,0,12/2,16.163534,48.333222,16.197313,48.310694
1,1,13/1,16.197252,48.333258,16.231016,48.310740
2,2,13/2,16.230970,48.333284,16.264719,48.310775
3,3,14/1,16.264688,48.333300,16.298423,48.310801
4,4,14/2,16.298406,48.333306,16.332126,48.310817
...,...,...,...,...,...,...
145,145,57/4,16.499997,48.130843,16.533653,48.108307
146,146,58/3,16.533567,48.130790,16.567239,48.108244
147,147,58/4,16.567138,48.130727,16.600824,48.108172
148,148,59/3,16.600708,48.130655,16.634409,48.108089
